# Fine-tuning de Modelos de Linguagem com Unsloth e LoRA

Este notebook demonstra como realizar fine-tuning em modelos de linguagem usando Unsloth com LoRA (Low-Rank Adaptation) para uma implementação mais eficiente.

In [ ]:
!pip install unsloth[cu118] -U
!pip install transformers datasets accelerate bitsandbytes

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from unsloth import FastLanguageModel
import os

# Definindo o caminho do workspace
WORKSPACE_PATH = "/content/drive/MyDrive/fiap/tech-challenge-03/"

# Configurações gerais
model_name = "meta-llama/Llama-2-7b-hf"  # Altere para o modelo desejado
max_length = 512
batch_size = 4
num_epochs = 1
learning_rate = 2e-4

In [ ]:
# Carregando o dataset
dataset = load_dataset('json', data_files={'train': f'{WORKSPACE_PATH}trn.json'})
dataset = dataset['train'].shuffle(seed=42).select(range(100000))  # Usando apenas 100k amostras

def prepare_data(examples):
    return {
        'text': [f"Title: {title}\nContent: {content}\nResponse:" for title, content in zip(examples['title'], examples['content'])]
    }

dataset = dataset.map(prepare_data, batched=True, remove_columns=dataset.column_names)
dataset = dataset.train_test_split(test_size=0.1)

print(f"Tamanho do conjunto de treinamento: {len(dataset['train'])}")
print(f"Tamanho do conjunto de validação: {len(dataset['test'])}")

In [ ]:
# Configuração do modelo e tokenizador com Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_length,
    dtype=None,  # Escolhe automaticamente o melhor dtype
    load_in_4bit=True,  # Carrega o modelo em 4-bit para economia de memória
)

# Adiciona LoRA ao modelo
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,  # Experimente com True para RsLoRA
)

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=max_length)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

In [ ]:
# Configuração do treinamento
training_args = TrainingArguments(
    output_dir=f'{WORKSPACE_PATH}results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=f'{WORKSPACE_PATH}logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    fp16=True,
    gradient_accumulation_steps=8,
    learning_rate=learning_rate,
    remove_unused_columns=False,  # Importante para Unsloth
)

# Configurando o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [ ]:
# Treinamento
trainer.train()

In [ ]:
# Salvando o modelo
output_dir = f"{WORKSPACE_PATH}unsloth_model_optimized"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
# Função para gerar respostas
def generate_response(title, content):
    input_text = f"Title: {title}\nContent: {content}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Response:')[-1].strip()
    
    return response

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Título: {title}")
print(f"Conteúdo: {content}")
print(f"Resposta gerada: {response}")